# 연습문제

In [1]:
import os
path = "C:/pytest/"
os.chdir(path)

In [2]:
import pandas as pd
import numpy as np

In [3]:
loan_o = pd.read_csv('./불법_대부업_데이터/class0(대출_합법).csv', encoding = 'cp949')
loan_o.head()

,Content,Label
0,그라나다 할부 조건 시간 전국 어디 중고차 할부 가능 여부 서비스 상담 신청 양식 ...,class0
1,광고 광고 올해 통신 인터넷 보이스 피싱 메모리 해킹 사기 피해 건수 피해액 최민희...,class0
2,춘천 연합뉴스 이해 기자 강원도 교육청 올해 보급 예정 교과서 전자 보이스 피싱 사...,class0
3,조선족 보이스 피싱,class0
4,조선족 보이스 피싱,class0


In [4]:
loan_x = pd.read_csv('./불법_대부업_데이터/class1(대출_불법).csv', encoding = 'cp949')
loan_x.head()

,Content,Label
0,저지 레드 다중 과다 채무자 증가 카드 대출 연체 증가,class1
1,히트 로드 채팅 싸이트 은행 신협 고객 연체 이자 부담,class1
2,보이스 피싱 아우 짜증 이제 그쪽 전화 보이스 피싱 진드기 사람 매트리스 사용,class1
3,보이스 피싱 아우 짜증 이제 그쪽 전화 보이스 피싱 진드기 사람 매트리스 사용,class1
4,보이스 피싱 아우 짜증 이제 그쪽 전화 보이스 피싱 진드기 사람 매트리스,class1


In [5]:
loan = pd.concat([loan_o,loan_x])
loan.head()

,Content,Label
0,그라나다 할부 조건 시간 전국 어디 중고차 할부 가능 여부 서비스 상담 신청 양식 ...,class0
1,광고 광고 올해 통신 인터넷 보이스 피싱 메모리 해킹 사기 피해 건수 피해액 최민희...,class0
2,춘천 연합뉴스 이해 기자 강원도 교육청 올해 보급 예정 교과서 전자 보이스 피싱 사...,class0
3,조선족 보이스 피싱,class0
4,조선족 보이스 피싱,class0


In [6]:
X = loan['Content']
y = loan['Label']

In [7]:
text_len = [len(line.split(' ')) for line in X]

In [8]:
print('최소길이 : {}\n최대길이 : {}\n평균길이 : {}\n중위수길이 : {}\n구간별 최대 길이 : {}\n최소길이 문장 : {}\n최대길이 문장 : {}'.format(np.min(text_len),np.max(text_len),np.round(np.mean(text_len),1), np.median(text_len), np.percentile(text_len, [0,25,50,75,99,100]), X[np.argmin(text_len)], X[np.argmax(text_len)]))

최소길이 : 1
최대길이 : 20
평균길이 : 12.7
중위수길이 : 11.0
구간별 최대 길이 : [ 1.  8. 11. 20. 20. 20.]
최소길이 문장 : 225                                                   충동
225    중고 중고 스마트폰 스마트폰 전기 연체 미납 해지 사용 경기 수도 인천 서울 고가 ...
Name: Content, dtype: object
최대길이 문장 : 0    그라나다 할부 조건 시간 전국 어디 중고차 할부 가능 여부 서비스 상담 신청 양식 ...
0                       저지 레드 다중 과다 채무자 증가 카드 대출 연체 증가
Name: Content, dtype: object


In [9]:
from sklearn.model_selection import train_test_split
train_X, test_X, train_y, test_y = train_test_split(X,y)

In [10]:
from sklearn.preprocessing import LabelEncoder
e = LabelEncoder()
e.fit(train_y)
train_y = e.transform(train_y)
train_y

array([0, 1, 0, ..., 0, 0, 0])

# Tokenizing

In [11]:
from keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences

In [12]:
max_words = 5000
maxlen = 20

In [13]:
tokenizer = Tokenizer(num_words= max_words)
tokenizer.fit_on_texts(train_X)

In [14]:
data_sequencing = tokenizer.texts_to_sequences(train_X)

# Padding

In [15]:
data_padding = pad_sequences(data_sequencing, maxlen= maxlen)

In [16]:
import math

In [17]:
def to_one_hot(sequences, dimension):
    results = np.zeros((len(sequences),dimension))
    for i, sequence in enumerate(sequences):
        results[i, sequence] = 1
    return results

In [18]:
data_1hot = to_one_hot(data_padding, dimension= max_words)
labels = np.array(train_y).astype('float32')

# 모델 설계

In [19]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense

In [20]:
model = Sequential()
model.add(Dense(64, activation = 'relu',input_dim =max_words))
model.add(Dense(32, activation = 'relu'))
model.add(Dense(16, activation = 'relu'))
model.add(Dense(8, activation = 'relu'))
model.add(Dense(1, activation = 'sigmoid'))
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense (Dense)               (None, 64)                320064    
                                                                 
 dense_1 (Dense)             (None, 32)                2080      
                                                                 
 dense_2 (Dense)             (None, 16)                528       
                                                                 
 dense_3 (Dense)             (None, 8)                 136       
                                                                 
 dense_4 (Dense)             (None, 1)                 9         
                                                                 
Total params: 322,817
Trainable params: 322,817
Non-trainable params: 0
_________________________________________________________________


In [21]:
# os.makedirs('C:/pytest/Problem/Problem1_20221007/')

In [22]:
from keras.callbacks import ModelCheckpoint
callbacks = ModelCheckpoint(
                    filepath = 'C:/pytest/Problem/Problem1_20221007/Problem1_model01.h5',
                    monitor = 'val_loss', verbose = 1, save_best_only= True)

In [23]:
model.compile(loss= 'binary_crossentropy', optimizer = 'adam', metrics = ['acc'])
history = model.fit(data_1hot, labels, epochs = 10, batch_size = 32, callbacks = [callbacks])
history_dict = history.history

Epoch 1/10
104/104 [==============================] - 2s 3ms/step - loss: 0.3041 - acc: 0.9539
Epoch 2/10
104/104 [==============================] - 0s 3ms/step - loss: 0.0271 - acc: 0.9906
Epoch 3/10
104/104 [==============================] - 0s 3ms/step - loss: 0.0068 - acc: 0.9976
Epoch 4/10
104/104 [==============================] - 0s 3ms/step - loss: 0.0024 - acc: 0.9994
Epoch 5/10
104/104 [==============================] - 0s 3ms/step - loss: 8.3352e-04 - acc: 1.0000
Epoch 6/10
104/104 [==============================] - 0s 4ms/step - loss: 4.5936e-04 - acc: 1.0000
Epoch 7/10
104/104 [==============================] - 0s 3ms/step - loss: 3.0198e-04 - acc: 1.0000
Epoch 8/10
104/104 [==============================] - 0s 3ms/step - loss: 2.0936e-04 - acc: 1.0000
Epoch 9/10
104/104 [==============================] - 0s 3ms/step - loss: 1.5178e-04 - acc: 1.0000
Epoch 10/10
104/104 [==============================] - 0s 3ms/step - loss: 1.1209e-04 - acc: 1.0000
